Notebook used to inspect results of Granger Causality analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import math
import sys
import re
from itertools import combinations
from matplotlib_venn import venn3

# Get the project root: notebooks/AI_narrative_index
root_dir = Path.cwd().parent

# Add needed folders to the Python modules search path
sys.path.append(str(root_dir / "src" / "scripts"))
sys.path.append(str(root_dir / "src" / "visualizations"))
sys.path.append(str(root_dir / "src" / "modelling"))

# import custom functions
#rom plot_granger_causality import plot_aini_lags_by_year, plot_aini_lags_for_year
from plot_functions import plot_n_articles_with_extrema_events, plot_stock_growth
from construct_tables import export_regression_table
from compute_extrema import compute_aini_extrema
from scipy.stats import pearsonr

No control

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c = pd.read_csv(var_path / "granger_causality_binary.csv")
gc_w0 = pd.read_csv(var_path / "granger_causality_w0.csv")
gc_w1 = pd.read_csv(var_path / "granger_causality_w1.csv")
gc_w2 = pd.read_csv(var_path / "granger_causality_w2.csv")


# create column to indicate version
gc_c["Model"] = "custom"
gc_w0["Model"] = "w0"
gc_w1["Model"] = "w1"
gc_w2["Model"] = "w2"

# merge them together
gc_all_results = pd.concat([gc_c, gc_w0, gc_w1, gc_w2], ignore_index=True)
gc_all_results["joint rej. (α=0.1)"] = gc_all_results["BH_reject_F"] & gc_all_results["BH_reject_F_HC3"]



In [ ]:
# ensure no dupls
seen = {}
new_cols = []

for c in gc_all_results.columns:
    if c not in seen:
        seen[c] = 0
        new_cols.append(c)
    else:
        seen[c] += 1
        new_cols.append(f"{c}.{seen[c]}")

gc_all_results.columns = new_cols

gc_all_results.columns

# HTML output
export_regression_table(
    df=gc_all_results,
    title="Granger-Causality all Results (AINI → Returns)",
    output_filename="gc_all_results_sox_cont",
    output_format="html"
)

In [ ]:
rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# Add lag-based renames (A2R and R2A)
for i in range(1, 4):
    rename_map[f"A2R_beta_ret_{i}"] = f"β{i}"
    rename_map[f"A2R_beta_x_{i}"] = f"γ{i}"
    rename_map[f"R2A_beta_ret_{i}"] = f"β{i}"
    rename_map[f"R2A_beta_x_{i}"] = f"γ{i}"

# Apply renaming
gc_all_results = gc_all_results.rename(columns=rename_map)
gc_all_results

# drop non-stationary measures, i.e. windows in 2025; EMA_{0.2} in 2025 for costum
gc_all_results_for_report = gc_all_results[(gc_all_results["Model"] == "custom") | (gc_all_results["Period"] != "2025")]
gc_all_results_for_report  = gc_all_results_for_report[~((gc_all_results_for_report["AINI_variant"] == "EMA_02") & (gc_all_results_for_report["Period"] == "2025"))]

print(len(gc_all_results_for_report) / 2)

In [ ]:
gc_all_results_for_report_a2r = gc_all_results_for_report[gc_all_results_for_report["Direction"] == "AINI_to_RET"]
gc_all_results_for_report_a2r = gc_all_results_for_report_a2r.dropna(axis=1, how='all')

# drop non-stationary measures, i.e. windows in 2025; EMA_{0.2} in 2025 for costum
gc_all_results_for_report_a2r

In [ ]:
gc_all_results


In [ ]:
# HTML output
export_regression_table(
    df=gc_all_results,
    title="Granger-Causality all Results (AINI → Returns)",
    output_filename="gc_sp500_aini_to_ret",
    output_format="html"
)

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
gc_all_results_for_report_a2r["Period"] = gc_all_results_for_report_a2r["Period"].astype(str)
gc_all_results_for_report_a2r["Ticker"] = gc_all_results_for_report_a2r["Ticker"].astype(str)

# Total number of models tested
total = gc_all_results_for_report_a2r["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = gc_all_results_for_report_a2r["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")

controlled for n art

In [ ]:

# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c = pd.read_csv(var_path / "granger_causality_log_growth_n_art_binary.csv")
gc_w0 = pd.read_csv(var_path / "granger_causality_log_growth_n_art_w0.csv")
gc_w1 = pd.read_csv(var_path / "granger_causality_log_growth_n_art_w1.csv")
gc_w2 = pd.read_csv(var_path / "granger_causality_log_growth_n_art_w2.csv")


# create column to indicate version
gc_c["Model"] = "custom"
gc_w0["Model"] = "w0"
gc_w1["Model"] = "w1"
gc_w2["Model"] = "w2"

# merge them together
gc_all_results_art = pd.concat([gc_c, gc_w0, gc_w1, gc_w2], ignore_index=True)
gc_all_results_art["joint rej. (α=0.1)"] = gc_all_results_art["BH_reject_F"] & gc_all_results_art["BH_reject_F_HC3"]
gc_all_results_art

In [ ]:
# ensure no dupls
seen = {}
new_cols = []

for c in gc_all_results_art.columns:
    if c not in seen:
        seen[c] = 0
        new_cols.append(c)
    else:
        seen[c] += 1
        new_cols.append(f"{c}.{seen[c]}")

gc_all_results_art.columns = new_cols

# HTML output
export_regression_table(
    df=gc_all_results_art,
    title="Granger-Causality all Results (controlled for log growth daily articles)",
    output_filename="gc_all_results_article_cont",
    output_format="html"
)

In [ ]:
gc_all_results_art.columns

Controlled for sox

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_binary.csv")
gc_w0_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_w0.csv")
gc_w1_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_w1.csv")
gc_w2_sox = pd.read_csv(var_path / "granger_causality_log_growth_sox_w2.csv")


# create column to indicate version
gc_c_sox["Model"] = "custom"
gc_w0_sox["Model"] = "w0"
gc_w1_sox["Model"] = "w1"
gc_w2_sox["Model"] = "w2"

rename_map = {
    # bookkeeping
    "p_x": "Lags",
    "Original_F_pval": "analytical P",
    "Empirical_F_pval": "empirical P",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period",

    # lagged return betas (AINI → Returns)
    "A2R_beta_ret_1": "β1",
    "A2R_beta_ret_2": "β2",
    "A2R_beta_ret_3": "β3",

    # AINI lag coefficients
    "A2R_beta_x_1": "γ1",
    "A2R_beta_x_2": "γ2",
    "A2R_beta_x_3": "γ3",

    # VIX control coefficients (original + .1 versions)
    "β_ctrl_log_growth_closed1": "ζ1",
    "β_ctrl_log_growth_closed2": "ζ2",
    "β_ctrl_log_growth_closed3": "ζ3",
    "β_ctrl_log_growth_closed1.1": "ζ1",
    "β_ctrl_log_growth_closed2.1": "ζ2",
    "β_ctrl_log_growth_closed3.1": "ζ3",

    # ---- Reverse direction (Returns → AINI) ----
    "R2A_beta_ret_1": "β1",
    "R2A_beta_ret_2": "β2",
    "R2A_beta_ret_3": "β3",

    "R2A_beta_x_1": "γ1",
    "R2A_beta_x_2": "γ2",
    "R2A_beta_x_3": "γ3",
}

rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# Add lag-based renames (A2R and R2A)
for i in range(1, 4):
    rename_map[f"A2R_beta_ret_{i}"] = f"β{i}"
    rename_map[f"A2R_beta_x_{i}"] = f"γ{i}"
    rename_map[f"R2A_beta_ret_{i}"] = f"β{i}"
    rename_map[f"R2A_beta_x_{i}"] = f"γ{i}"


# merge them together
gc_all_results_sox = pd.concat([gc_c_sox, gc_w0_sox, gc_w1_sox, gc_w2_sox], ignore_index=True)
gc_all_results_sox["joint rej. (α=0.1)"] = gc_all_results_sox["BH_reject_F"] & gc_all_results_sox["BH_reject_F_HC3"]

# Apply renaming
gc_all_results_sox = gc_all_results_sox.rename(columns=rename_map)
gc_all_results_sox
gc_all_results_sox.columns

In [ ]:
# ensure no dupls
seen = {}
new_cols = []

for c in gc_all_results_sox.columns:
    if c not in seen:
        seen[c] = 0
        new_cols.append(c)
    else:
        seen[c] += 1
        new_cols.append(f"{c}.{seen[c]}")

gc_all_results_sox.columns = new_cols

gc_all_results_sox.columns

# HTML output
export_regression_table(
    df=gc_all_results_sox,
    title="Granger-Causality all Results (controlled for SOX)",
    output_filename="gc_all_results_sox_cont",
    output_format="html"
)

In [ ]:
# Apply renaming
gc_all_results_sox = gc_all_results_sox.rename(columns=rename_map)
gc_all_results_sox

gc_all_results_for_report_sox = gc_all_results_sox.copy()
gc_all_results_for_report_sox

# drop non-stationary measures, i.e. windows in 2025; EMA_{0.2} in 2025 for costum
gc_all_results_for_report_sox = gc_all_results_for_report_sox[(gc_all_results_for_report_sox["Model"] == "custom") | (gc_all_results_for_report_sox["Period"] != "2025")]
gc_all_results_for_report_sox  = gc_all_results_for_report_sox[~((gc_all_results_for_report_sox["AINI_variant"] == "EMA_02") & (gc_all_results_for_report_sox["Period"] == "2025"))]

print(len(gc_all_results_for_report_sox) / 2)
gc_c_sox

In [ ]:
cols = ["AINI_variant", "Ticker", "Period", "Model"]

# ensure string dtype for merge keys
for c in cols:
    gc_all_results[c] = gc_all_results[c].astype(str)
    gc_all_results_sox[c] = gc_all_results_sox[c].astype(str)

# ensure subset of relevant columns exists in both DataFrames
missing_rows = gc_all_results.merge(
    gc_all_results_sox[cols],
    on=cols,
    how="left",
    indicator=True
).query('_merge == "left_only"') \
 .drop(columns="_merge")

# display or save
print(missing_rows)

In [ ]:
gc_all_results_for_report_a2r_sox = gc_all_results_for_report_sox[gc_all_results_for_report_sox["Direction"] == "AINI_to_RET"]
gc_all_results_for_report_a2r_sox = gc_all_results_for_report_a2r_sox.dropna(axis=1, how='all')
gc_all_results_for_report_a2r_sox

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
gc_all_results_for_report_a2r_sox["Period"] = gc_all_results_for_report_a2r_sox["Period"].astype(str)
gc_all_results_for_report_a2r_sox["Ticker"] = gc_all_results_for_report_a2r_sox["Ticker"].astype(str)

# Total number of models tested
total = gc_all_results_for_report_a2r_sox["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = gc_all_results_for_report_a2r_sox["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")

Controlled for S&P 500

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c = pd.read_csv(var_path / "granger_causality_log_growth_sp500_binary.csv")
gc_w0 = pd.read_csv(var_path / "granger_causality_log_growth_sp500_w0.csv")
gc_w1 = pd.read_csv(var_path / "granger_causality_log_growth_sp500_w1.csv")
gc_w2 = pd.read_csv(var_path / "granger_causality_log_growth_sp500_w2.csv")


# create column to indicate version
gc_c["Model"] = "custom"
gc_w0["Model"] = "w0"
gc_w1["Model"] = "w1"
gc_w2["Model"] = "w2"

# merge them together
gc_all_results = pd.concat([gc_c, gc_w0, gc_w1, gc_w2], ignore_index=True)
gc_all_results_sp500 = gc_all_results.copy()
gc_all_results_sp500["joint rej. (α=0.1)"] = gc_all_results["BH_reject_F"] & gc_all_results["BH_reject_F_HC3"]
gc_all_results_sp500

In [ ]:
# ensure no dupls
seen = {}
new_cols = []

for c in gc_all_results_sp500.columns:
    if c not in seen:
        seen[c] = 0
        new_cols.append(c)
    else:
        seen[c] += 1
        new_cols.append(f"{c}.{seen[c]}")

gc_all_results_sp500.columns = new_cols

gc_all_results_sp500.columns

# HTML output
export_regression_table(
    df=gc_all_results_sp500,
    title="Granger-Causality all Results (controlled for S&P500)",
    output_filename="gc_all_results_sp500_cont",
    output_format="html"
)

In [ ]:
rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# Add lag-based renames (A2R and R2A)
for i in range(1, 4):
    rename_map[f"A2R_beta_ret_{i}"] = f"β{i}"
    rename_map[f"A2R_beta_x_{i}"] = f"γ{i}"
    rename_map[f"R2A_beta_ret_{i}"] = f"β{i}"
    rename_map[f"R2A_beta_x_{i}"] = f"γ{i}"

# Apply renaming
gc_all_results_sp500 = gc_all_results.rename(columns=rename_map)
gc_all_results_sp500

# drop non-stationary measures, i.e. windows in 2025; EMA_{0.2} in 2025 for costum
gc_all_results_sp500 = gc_all_results_sp500[(gc_all_results_sp500["Model"] == "custom") | (gc_all_results_sp500["Period"] != "2025")]
gc_all_results_sp500  = gc_all_results_sp500[~((gc_all_results_sp500["AINI_variant"] == "EMA_02") & (gc_all_results_sp500["Period"] == "2025"))]

gc_all_results_sp500_for_report = gc_all_results_sp500.copy()

print(len(gc_all_results_sp500_for_report) / 2)

gc_all_results_sp500_for_report_a2r = gc_all_results_sp500_for_report[gc_all_results_sp500_for_report["Direction"] == "AINI_to_RET"]
gc_all_results_sp500_for_report_a2r = gc_all_results_sp500_for_report_a2r.dropna(axis=1, how='all')
gc_all_results_sp500_for_report_a2r

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
gc_all_results_sp500_for_report_a2r["Period"] = gc_all_results_sp500_for_report_a2r["Period"].astype(str)
gc_all_results_sp500_for_report_a2r["Ticker"] = gc_all_results_sp500_for_report_a2r["Ticker"].astype(str)

# Total number of models tested
gc_all_results_sp500_for_report_a2r["joint rej. (α=0.1)"] = gc_all_results_sp500_for_report_a2r["BH_reject_F"] & gc_all_results_sp500_for_report_a2r["BH_reject_F_HC3"]

total = gc_all_results_sp500_for_report_a2r["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = gc_all_results_sp500_for_report_a2r["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")
gc_all_results_for_report_a2r

In [ ]:
# find divergence no control vs S&P 500

# Define key columns
keys = ["Ticker", "AINI_variant", "Period","Lags"]

# Filter both for joint rejections
a_sig = gc_all_results_sp500_for_report_a2r.loc[
    gc_all_results_sp500_for_report_a2r["joint rej. (α=0.1)"] == True, keys
]
b_sig = gc_all_results_for_report_a2r.loc[
    gc_all_results_for_report_a2r["joint rej. (α=0.1)"] == True, keys
]

# Convert to sets of unique tuples
set_a = set(map(tuple, a_sig.drop_duplicates().to_numpy()))
set_b = set(map(tuple, b_sig.drop_duplicates().to_numpy()))

# Compute differences
only_in_a = set_a - set_b
only_in_b = set_b - set_a
common = set_a & set_b

# Quick stats
print(f"Unique in SP500 (only_in_a): {len(only_in_a)}")
print(f"Unique in All (only_in_b):   {len(only_in_b)}")
print(f"Common significant pairs:    {len(common)}")

# Optional DataFrames for inspection
only_in_a_df = pd.DataFrame(list(only_in_a), columns=keys)
only_in_b_df = pd.DataFrame(list(only_in_b), columns=keys)
only_in_a_df 

In [ ]:
# HTML output
export_regression_table(
    df=gc_all_results_sp500,
    title="Granger-Causality all Results (AINI → Returns, controlled for S&P500)",
    output_filename="gc_all_results_sp500_cont",
    output_format="html"
)

In [ ]:
# indicate significance
gc_all_results_sp500["joint rej. (α=0.1)"] = gc_all_results_sp500["BH_reject_F"] & gc_all_results_sp500["BH_reject_F_HC3"]

In [ ]:
# subset by direction
sp500_aini_to_ret = gc_all_results_sp500[gc_all_results_sp500["Direction"] == "AINI_to_RET"]
sp500_ret_to_aini = gc_all_results_sp500[gc_all_results_sp500["Direction"] == "RET_to_AINI"]

# cols to keep
keep_a2r = [
    "Model", "AINI_variant","adj_r2_u", "Ticker", "Period", "Lags",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)"
]

keep_r2a = [
    "Model", "AINI_variant","adj_r2_u", "Ticker", "Period", "Lags",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)"
]

# subset
sp500_aini_to_ret_sub = sp500_aini_to_ret[keep_a2r]
sp500_ret_to_aini_sub = sp500_ret_to_aini[keep_r2a]

sp500_aini_to_ret.columns

In [ ]:
sp500_aini_to_ret_sub

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
sp500_aini_to_ret_sub["Period"] = sp500_aini_to_ret_sub["Period"].astype(str)
sp500_aini_to_ret_sub["Ticker"] = sp500_aini_to_ret_sub["Ticker"].astype(str)

# 

# Total number of models tested
total = sp500_aini_to_ret_sub["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = sp500_aini_to_ret_sub["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")


In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
sp500_ret_to_aini_sub["Period"] = sp500_ret_to_aini_sub["Period"].astype(str)
sp500_ret_to_aini_sub["Ticker"] = sp500_ret_to_aini_sub["Ticker"].astype(str)

# Total number of models tested
total = sp500_ret_to_aini_sub["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = sp500_ret_to_aini_sub["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")
sp500_ret_to_aini_sub

In [ ]:
# subset significant results
sp500_aini_to_ret_sig = sp500_aini_to_ret_sub[sp500_aini_to_ret_sub["joint rej. (α=0.1)"] == True]
sp500_ret_to_aini_sig = sp500_ret_to_aini_sub[sp500_ret_to_aini_sub["joint rej. (α=0.1)"] == True]
# drop 0 value cols
sp500_ret_to_aini_sig = sp500_ret_to_aini_sig.dropna(axis=1,how="all")

# Coerce β columns to numeric 
sp500_ret_to_aini_sig[["β1","β2","β3"]] = (
    sp500_ret_to_aini_sig[["β1","β2","β3"]]
    .apply(pd.to_numeric, errors="coerce")
)
sp500_ret_to_aini_sig

In [ ]:
df = sp500_aini_to_ret_sig.copy()
df["Period"] = df["Period"].replace({
    "2023_24_25": "2023-2025",
    "2024_25": "2024-2025",
    "2023_24": "2023-2024"
}).astype(str)
df["Ticker"] = df["Ticker"].astype(str)

#  build order 
def period_key(p: str):
    years = list(map(int, re.findall(r"\d{4}", p)))
    if len(years) == 1:
        years = [years[0], years[0]]
    start, end = years[0], years[-1]
    return (end, start) 

periods = sorted(df["Period"].unique(), key=period_key, reverse=True)
df["Period"] = pd.Categorical(df["Period"], categories=periods, ordered=True)

# plot 
sns.set_theme(style="whitegrid", font_scale=1.2)
plt.figure(figsize=(10, 5), dpi=300)

ax = sns.countplot(
    data=df,
    x="Ticker",
    hue="Period",
    hue_order=periods,                          
    palette=sns.color_palette("viridis", n_colors=len(periods)),
    edgecolor="black"
)

ax.set_title("Significant results by Ticker and Period", fontsize=13, pad=10)
ax.set_xlabel("Ticker", fontsize=12)
ax.set_ylabel("Count of Significant Results", fontsize=12)
ax.legend(title="Period", title_fontsize=11, fontsize=10, loc="upper right", frameon=True)

plt.xticks(rotation=30, ha="right")
sns.despine(trim=True)
plt.tight_layout()
plt.savefig(root_dir / "reports/figures/sp500_aini_to_ret_sig_counts.png", dpi=600, bbox_inches="tight")
plt.show()


In [ ]:
# group by model
model_group_tickers = (
    sp500_aini_to_ret_sig
    .groupby(["Ticker"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_tickers)

In [ ]:
# group by model all
model_group_tickers = (
    sp500_aini_to_ret
    .groupby(["Model"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_tickers)

In [ ]:
# group by period
model_group_period = (
    sp500_aini_to_ret_sig
    .groupby(["Period"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_period)


In [ ]:
# group by period total
model_group_period = (
    sp500_aini_to_ret
    .groupby(["Period"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_period)

In [ ]:
# group by Ticker
model_group_period = (
    sp500_aini_to_ret_sig
    .groupby(["Ticker"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_period)

In [ ]:
# group by Ticker total
model_group_period = (
    sp500_aini_to_ret
    .groupby(["Ticker"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
sp500_aini_to_ret

In [ ]:
# group by period & Ticker
model_group_tickers = (
    sp500_aini_to_ret_sig
    .groupby(["Model"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
)


# print
print(model_group_tickers)

In [ ]:
# group by variant
model_group_measure = (
    sp500_aini_to_ret_sig
    .groupby(["AINI_variant"])
    .size()
    .reset_index(name="n_variants")
    .sort_values(by="n_variants",ascending=False)
)

print(model_group_measure)

In [ ]:
# find distinctions between models
keys = ["Ticker", "Period"]  
models = ["w0", "w1", "w2", "custom"]               

common_dfs = []
left_only_dfs = []
right_only_dfs = []

for m1, m2 in combinations(models, 2):
    df1 = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == m1, keys].drop_duplicates()
    df2 = sp500_aini_to_ret_sig.loc[sp500_aini_to_ret_sig["Model"] == m2, keys].drop_duplicates()

    # intersection
    common = df1.merge(df2, on=keys, how="inner")
    if not common.empty:
        common = common.assign(Model_pair=f"{m1}&{m2}")
        common_dfs.append(common)

    # only in left / only in right
    cmp = df1.merge(df2, on=keys, how="outer", indicator=True)
    left_only  = cmp.loc[cmp["_merge"] == "left_only",  keys].assign(only=m1)
    right_only = cmp.loc[cmp["_merge"] == "right_only", keys].assign(only=m2)

    if not left_only.empty:
        left_only_dfs.append(left_only)
    if not right_only.empty:
        right_only_dfs.append(right_only)

# Concatenate
common_all = pd.concat(common_dfs, ignore_index=True) if common_dfs else pd.DataFrame(columns=keys+["Model_pair"])
left_only_all = pd.concat(left_only_dfs, ignore_index=True) if left_only_dfs else pd.DataFrame(columns=keys+["only"])
right_only_all = pd.concat(right_only_dfs, ignore_index=True) if right_only_dfs else pd.DataFrame(columns=keys+["only"])

right_only_all

In [ ]:
keys = ["Ticker", "Period", "Model"]

# Defensive copies (optional but clean)
A = sp500_aini_to_ret_sig.copy()
B = sp500_ret_to_aini_sig.copy()

# Normalize data types
for c in keys:
    A[c] = A[c].astype(str)
    B[c] = B[c].astype(str)

# Build sets of unique tuples
S_A = set(map(tuple, A[keys].drop_duplicates().to_numpy()))
S_B = set(map(tuple, B[keys].drop_duplicates().to_numpy()))

# Compute intersection and counts
intersection = S_A & S_B
n_inter = len(intersection)

print(f"Common observations (unique {keys}): {n_inter}")
print(f"|A| = {len(S_A)}, |B| = {len(S_B)}, overlap = {n_inter / len(S_A):.2%} of A, {n_inter / len(S_B):.2%} of B")


In [ ]:
# Jaccard for aini-Ret; ret -> AINI sp500_ret_to_aini_sig

In [ ]:
# investigate groups by Model
model_group_model = (
    sp500_aini_to_ret_sig
    .groupby(["Model"])
    .size()
    .reset_index(name="n_variants")
    .sort_values(by="n_variants",ascending=False)
)
print(model_group_model)


In [ ]:
# beautify
sp500_aini_to_ret_sig["Period"] = sp500_aini_to_ret_sig["Period"].replace({"2023_24": "2023-2024"})
sp500_aini_to_ret_sig["Period"] = sp500_aini_to_ret_sig["Period"].replace({"2024_25": "2024-2025"})
sp500_aini_to_ret_sig["Period"] = sp500_aini_to_ret_sig["Period"].replace({"2023_24_25": "2023-2025"})

In [ ]:
# sort by betas
sp500_aini_to_ret_sig = sp500_aini_to_ret_sig.dropna(axis=1,how="all")

sp500_aini_to_ret_sort = sp500_aini_to_ret_sig.assign(abs_β1=lambda x: x["β1"].abs()).sort_values("abs_β1", ascending=False)
sp500_aini_to_ret_sort_cut = sp500_aini_to_ret_sort.iloc[0:20]

# inspect 
sp500_aini_to_ret_sort_cut

In [ ]:
# sorted latex output
export_regression_table(
    df=sp500_aini_to_ret_sort,
    title="Granger-Causality, jointly significant results (AINI $\\to$ Returns, controlled for S\\&P~500). \\textit{Source:} Own.",
    output_filename="gc_sp500_aini_to_ret_sort_beta",  
    output_format="tex",
    latex_env="tabular",          
    include_caption_label=False,    
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",   
)


In [ ]:
export_regression_table(
    df=sp500_aini_to_ret_sort,
    title="Granger-Causality, jointly significant results (AINI → Returns, controlled for S&P 500)",
    output_filename="gc_sp500_aini_to_ret_sort_beta",
    output_format="tex",
    latex_env="longtable",         
    include_caption_label=True,    
    font_size_cmd="scriptsize",
    tabcolsep_pt=2.0,
    coef_digits=3,
    p_digits=3
)


In [ ]:
# beautify opposite direction 
sp500_ret_to_aini_sig["Period"] = sp500_ret_to_aini_sig["Period"].replace({"2023_24": "2023-2024"})
sp500_ret_to_aini_sig["Period"] = sp500_ret_to_aini_sig["Period"].replace({"2024_25": "2024-2025"})
sp500_ret_to_aini_sig["Period"] = sp500_ret_to_aini_sig["Period"].replace({"2023_24_25": "2023-2025"})

# drop NA
sp500_ret_to_aini_sig = sp500_ret_to_aini_sig.dropna(axis=1, how="all")

# sort by betas
sp500_ret_to_aini_sig_sort = sp500_ret_to_aini_sig.assign(abs_γ1=lambda x: x["γ1"].abs()).sort_values("abs_γ1", ascending=False)
sp500_ret_to_aini_sig_sort_cut = sp500_ret_to_aini_sig_sort.iloc[0:20]
sp500_ret_to_aini_sig

In [ ]:
export_regression_table(
    df=sp500_ret_to_aini_sig_sort_cut,
    title="Granger-Causality, jointly significant results (Returns -> AINI, controlled for S&P 500), Top 20",
    output_filename="gc_sp500_ret_to_aini_sort_beta_cut",
    output_format="tex",
    latex_env="tabular",           
    include_caption_label=False,   
    font_size_cmd="scriptsize",
    tabcolsep_pt=2.0
)
sp500_ret_to_aini_sig_sort

In [ ]:
export_regression_table(
    df=sp500_ret_to_aini_sig_sort,
    title="Granger-Causality, jointly significant results (Returns -> AINI, controlled for S&P 500), Top 20",
    output_filename="gc_sp500_ret_to_aini_sort_beta",
    output_format="tex",
    latex_env="tabular",           
    include_caption_label=False,   
    font_size_cmd="scriptsize",
    tabcolsep_pt=2.0
)

Controlling for VIX

In [ ]:
# define path to variables
var_path = root_dir / "data" / "processed" / "variables"
 
# load data (S&P 500 control)
gc_c = pd.read_csv(var_path / "granger_causality_VIX_binary.csv")
gc_w0 = pd.read_csv(var_path / "granger_causality_VIX_w0.csv")
gc_w1 = pd.read_csv(var_path / "granger_causality_VIX_w1.csv")
gc_w2 = pd.read_csv(var_path / "granger_causality_VIX_w2.csv")


# create column to indicate version
gc_c["Model"] = "custom"
gc_w0["Model"] = "w0"
gc_w1["Model"] = "w1"
gc_w2["Model"] = "w2"

# merge them together
gc_all_results_VIX = pd.concat([gc_c, gc_w0, gc_w1, gc_w2], ignore_index=True)
gc_all_results_VIX["joint rej. (α=0.1)"] = gc_all_results_VIX["BH_reject_F"] & gc_all_results_VIX["BH_reject_F_HC3"]


In [ ]:
# ensure no dupls
seen = {}
new_cols = []

for c in gc_all_results_VIX.columns:
    if c not in seen:
        seen[c] = 0
        new_cols.append(c)
    else:
        seen[c] += 1
        new_cols.append(f"{c}.{seen[c]}")

gc_all_results_VIX.columns = new_cols


# HTML output
export_regression_table(
    df=gc_all_results_VIX,
    title="Granger-Causality all Results (controlled for VIX)",
    output_filename="gc_all_results_vix_cont",
    output_format="html"
)

In [ ]:
gc_all_results_VIX_for_report = gc_all_results_VIX.copy()

# drop non-stationary measures, i.e. windows in 2025; EMA_{0.2} in 2025 for costum
gc_all_results_VIX_for_report = gc_all_results_VIX_for_report[(gc_all_results_VIX_for_report["Model"] == "custom") | (gc_all_results_VIX_for_report["Year"] != "2025")]
gc_all_results_VIX_for_report  = gc_all_results_VIX_for_report[~((gc_all_results_VIX_for_report["AINI_variant"] == "EMA_02") & (gc_all_results_VIX_for_report["Year"] == "2025"))]

# split by direction
vix_aini_to_ret = gc_all_results_VIX_for_report[gc_all_results_VIX_for_report["Direction"] == "AINI_to_RET"]
vix_ret_to_aini = gc_all_results_VIX_for_report[gc_all_results_VIX_for_report["Direction"] == "RET_to_AINI"]

# beautify
rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period"
}

# drop NAs
vix_ret_to_aini = vix_ret_to_aini.dropna(how="all",axis=1) 
vix_aini_to_ret = vix_aini_to_ret.dropna(how="all",axis=1) 
vix_ret_to_aini


In [ ]:
rename_map = {
    # bookkeeping
    "p_x": "Lags",
    "Original_F_pval": "analytical P",
    "Empirical_F_pval": "empirical P",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period",

    # lagged return betas (AINI → Returns)
    "A2R_beta_ret_1": "β1",
    "A2R_beta_ret_2": "β2",
    "A2R_beta_ret_3": "β3",

    # AINI lag coefficients
    "A2R_beta_x_1": "γ1",
    "A2R_beta_x_2": "γ2",
    "A2R_beta_x_3": "γ3",

    # VIX control coefficients (original + .1 versions)
    "β_ctrl_log_growth_closed1": "ζ1",
    "β_ctrl_log_growth_closed2": "ζ2",
    "β_ctrl_log_growth_closed3": "ζ3",
    "β_ctrl_log_growth_closed1.1": "ζ1",
    "β_ctrl_log_growth_closed2.1": "ζ2",
    "β_ctrl_log_growth_closed3.1": "ζ3",

    # ---- Reverse direction (Returns → AINI) ----
    "R2A_beta_ret_1": "β1",
    "R2A_beta_ret_2": "β2",
    "R2A_beta_ret_3": "β3",

    "R2A_beta_x_1": "γ1",
    "R2A_beta_x_2": "γ2",
    "R2A_beta_x_3": "γ3",

    "R2A_BH_corr_F_pval": "BH empirical p",
    "R2A_BH_corr_F_pval_HC3": "BH analytical p",
    "R2A_Empirical_F_pval": "empirical P",
    "R2A_Original_F_pval": "analytical P",
}



vix_aini_to_ret = vix_aini_to_ret.rename(columns=rename_map)
vix_ret_to_aini= vix_ret_to_aini.rename(columns=rename_map)

# sanity check: no duplicate column names
dup_mask = pd.Series(vix_aini_to_ret.columns).duplicated(keep=False)
if dup_mask.any():
    dups = pd.Series(vix_aini_to_ret.columns)[dup_mask].tolist()
    raise ValueError(f"Duplicate column names after rename: {dups}")


# Apply renaming
gc_all_results_VIX_for_report= vix_aini_to_ret.rename(columns=rename_map)

# inspect
vix_aini_to_ret.columns

In [ ]:
# cols to keep
keep_a2r = [
    "Model", "AINI_variant", "Ticker", "Period", "Lags",
    # Coefficients
    "β1", "β2", "β3",      # lagged returns
    "γ1", "γ2", "γ3",      # AINI lags
    "ζ1", "ζ2", "ζ3",      # VIX controls (
    # Fit quality
    "r2_u", "adj_r2_u",
    # P-values and test results
    "analytical P","empirical P",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)",
]


# Columns to keep for Return → AINI 
keep_r2a = [
    "Model", "AINI_variant", "Ticker", "Period", "Lags",
    "β1", "β2", "β3",
    "γ1", "γ2", "γ3",
    "ζ1", "ζ2", "ζ3", 
    "r2_u", "adj_r2_u",
    "analytical P","empirical P",
    "BH empirical p", "BH analytical p", "joint rej. (α=0.1)",
]

# Apply safely
vix_aini_to_ret = vix_aini_to_ret[[c for c in keep_a2r if c in vix_aini_to_ret.columns]]
vix_ret_to_aini = vix_ret_to_aini[[c for c in keep_r2a if c in vix_ret_to_aini.columns]]


# subset by col
vix_ret_to_aini_sub = vix_ret_to_aini[keep_r2a]
vix_aini_to_ret_sub = vix_aini_to_ret[keep_a2r]

# subset signficant
vix_ret_to_aini_sub = vix_ret_to_aini_sub[vix_ret_to_aini_sub["joint rej. (α=0.1)"] == True ]
vix_aini_to_ret_sub = vix_aini_to_ret_sub[vix_aini_to_ret_sub["joint rej. (α=0.1)"] == True ]

# drop duplicated cols
vix_ret_to_aini_sub = vix_ret_to_aini_sub.dropna(how="all",axis=1)
vix_aini_to_ret_sub = vix_aini_to_ret_sub.dropna(how="all",axis=1)



In [ ]:
vix_aini_to_ret_sub.columns

In [ ]:
# calculate rejection rate 

# Make sure Year and Ticker are strings
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].astype(str)
vix_aini_to_ret_sub["Ticker"] = vix_aini_to_ret_sub["Ticker"].astype(str)

# Total number of models tested
total = vix_ret_to_aini["joint rej. (α=0.1)"].count()

# Number of rejections (both bootstrap + HC3 significant)
n_reject = vix_ret_to_aini["joint rej. (α=0.1)"].sum()

# Rejection rate
rejection_rate = n_reject / total * 100

print(f"Total models: {total}")
print(f"Both-method rejections: {n_reject}")
print(f"Rejection rate: {rejection_rate:.2f}%")

In [ ]:
# subset
VIX_aini_to_ret_sig = vix_aini_to_ret_sub[vix_aini_to_ret_sub["joint rej. (α=0.1)"] == True ]

# beautify periods
p_rename_map = {
    "2023_24" : "2023-2024",
    "2023_24_25" : "2023-2025",
    "2024_25" : "2024-2025"
}

VIX_aini_to_ret_sig["Period"] = VIX_aini_to_ret_sig["Period"].replace(p_rename_map)
VIX_aini_to_ret_sig

In [ ]:
# ivnestigate difference to VIX

# Define the key columns
keys = ["Period", "AINI_variant", "Ticker","Lags","Model"]

# Defensive copies (optional but clean)
A = VIX_aini_to_ret_sig[keys].drop_duplicates().copy()
B = sp500_aini_to_ret_sig[keys].drop_duplicates().copy()

# Intersection
intersection = pd.merge(A, B, on=keys, how="inner")

# Unique to VIX (A)
unique_vix = pd.merge(A, B, on=keys, how="left", indicator=True).query('_merge == "left_only"').drop(columns="_merge")

# Unique to SP500 (B)
unique_sp500 = pd.merge(A, B, on=keys, how="right", indicator=True).query('_merge == "right_only"').drop(columns="_merge")

# Print summary
print(f"Intersection: {len(intersection)}")
print(f"Unique to VIX: {len(unique_vix)}")
print(f"Unique to SP500: {len(unique_sp500)}")

# total sp500  104; total VIX 

# (Optional) show proportions for context
print(f"Overlap rate (of VIX): {len(intersection)/len(A):.2%}")
VIX_aini_to_ret_sig.columns

In [ ]:
# ivnestigate difference to RET-AINI

# Define the key columns
keys = ["Period", "Ticker"]

# copies 
A = VIX_aini_to_ret_sig[keys].drop_duplicates().copy()
B = vix_ret_to_aini_sub[keys].drop_duplicates().copy()

# Intersection
intersection = pd.merge(A, B, on=keys, how="inner")

# Unique to VIX (A)
unique_vix = pd.merge(A, B, on=keys, how="left", indicator=True).query('_merge == "left_only"').drop(columns="_merge")

# Unique to SP500 (B)
unique_ret = pd.merge(A, B, on=keys, how="right", indicator=True).query('_merge == "right_only"').drop(columns="_merge")

# Print summary
print(f"Intersection: {len(intersection)}")
print(f"Unique to Ret->AINI: {len(unique_vix)}")
print(f"Unique to AINI->AINI: {len(unique_ret)}")

# show proportions for context
print(f"Overlap rate (of VIX): {len(intersection)/len(VIX_aini_to_ret_sig):.2%}")
VIX_aini_to_ret_sig.columns
intersection

In [ ]:
# plotting
out_path = root_dir / "reports" / "figures" / "distribution_of_gammas.png"
out_path.parent.mkdir(parents=True, exist_ok=True)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 10,
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
})

SUBSCRIPT_DIGITS = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789")

def norm_g(c):
    s = str(c).translate(SUBSCRIPT_DIGITS).replace("'", "").strip().lower()
    if s.startswith("gamma") or s.startswith("g"):
        s = "γ" + re.sub(r"^[a-z]+", "", s)
    return s

def coerce_numeric_df(df):
    cleaned = df.replace(r"[^0-9eE\.\+\-]", "", regex=True)
    return cleaned.apply(pd.to_numeric, errors="coerce")

wanted = {"γ1", "γ2", "γ3"}
groups = {"γ1": [], "γ2": [], "γ3": []}
for c in VIX_aini_to_ret_sig.columns:
    key = norm_g(c)
    if key in wanted:
        groups[key].append(c)

collapsed = pd.DataFrame(index=VIX_aini_to_ret_sig.index)
for key, cols in groups.items():
    collapsed[key] = coerce_numeric_df(VIX_aini_to_ret_sig[cols]).mean(axis=1) if cols else np.nan

all_gamma_vals = collapsed[["γ1","γ2","γ3"]].to_numpy(dtype=float).ravel()
all_gamma_vals = all_gamma_vals[~np.isnan(all_gamma_vals)]
lim = float(max(abs(all_gamma_vals.min()), abs(all_gamma_vals.max()))) if all_gamma_vals.size else 1.0
xlim = (-lim, lim)

models = ["w0", "w1", "w2", "custom"]
bins, ymax = 40, 27
fig, axes = plt.subplots(2, 2, figsize=(7.2, 5.6), sharex=True, sharey=True)
axes = axes.flatten()

legend_handles, legend_labels = None, None

for i, model in enumerate(models):
    ax = axes[i]
    subset = collapsed.loc[VIX_aini_to_ret_sig["Model"] == model, ["γ1","γ2","γ3"]].dropna(how="all", axis=1)

    if subset.empty:
        ax.set_title(f"Model: {model} (no data)")
        ax.set_xlim(xlim); ax.set_ylim(0, ymax)
        ax.grid(alpha=0.25, linestyle=":", linewidth=0.8)
        continue

    plot_ax = subset.plot.hist(
        bins=bins, range=xlim, alpha=0.6, ax=ax,
        edgecolor="black", legend=True, grid=False,
    )

    if legend_handles is None:
        legend_handles, legend_labels = plot_ax.get_legend_handles_labels()

    ax.set_title(f"Model: {model}")
    ax.set_xlim(xlim); ax.set_ylim(0, ymax)
    ax.set_ylabel("Counts")
    ax.grid(alpha=0.25, linestyle=":", linewidth=0.8)

    means = subset.mean().values
    labels = [fr"$\bar{{\gamma}}_{{{j}}}$={m:.3f}" for j, m in enumerate(means, start=1)]
    ax.text(
        0.98, 0.97, ", ".join(labels),
        transform=ax.transAxes, ha="right", va="top", fontsize=8.5,
        bbox=dict(facecolor="white", edgecolor="none", alpha=0.7, boxstyle="round,pad=0.2")
    )

    leg = ax.get_legend()
    if leg is not None:
        leg.remove()

fig.text(0.5, 0.02, "", ha="center", fontsize=10)

if legend_handles:
    fig.legend(
        legend_handles, legend_labels, title="Coefficient",
        loc="lower center", ncol=3, frameon=False
    )

plt.tight_layout(rect=[0.04, 0.07, 1, 0.98])
fig.savefig(out_path, bbox_inches="tight", facecolor="white")
plt.close(fig)
print(f"Saved: {out_path}")


In [ ]:
# group by model
model_group_tickers = (
    VIX_aini_to_ret_sig
    .groupby(["Ticker","Period"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_tickers)

In [ ]:
# group by ticker
model_group_tickers = (
    VIX_aini_to_ret_sig
    .groupby(["Ticker"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_tickers)



In [ ]:
# group by model
model_group_tickers = (
    sp500_aini_to_ret_sig
    .groupby(["Model"])
    .size()
    .reset_index(name="jointly rejected at α=0.1")
    .sort_values(by="jointly rejected at α=0.1",ascending=False)
)


# print
print(model_group_tickers)

In [ ]:
# Start from  dataframe
df = VIX_aini_to_ret_sig.copy()

# Isolate the duplicate γ1 columns and drop those that are all zeros ---
gamma1_block = df.loc[:, df.columns == "γ1"]  # this returns a DataFrame if there are duplicates
gamma1_keep = gamma1_block.loc[:, (gamma1_block != 0).any(axis=0)]

# keep the first non-zero per row
if gamma1_keep.shape[1] == 0:
    raise ValueError("No non-zero γ1 column found.")
elif gamma1_keep.shape[1] == 1:
    gamma1 = gamma1_keep.iloc[:, 0]
else:
    # Row-wise pick the first non-zero; fallback to the first column if all zeros (shouldn't happen after filtering)
    gamma1 = gamma1_keep.apply(lambda r: next((v for v in r if v != 0), r.iloc[0]), axis=1)

# Build a tidy table: one γ1 per (Period, Ticker, Lags, Model) ---
needed = df[["Period", "Ticker", "Lags", "Model"]].reset_index(drop=True)
gamma1 = gamma1.reset_index(drop=True).rename("γ1")
tidy = pd.concat([needed, gamma1], axis=1)

# Pivot: rows = (Period, Ticker, Lags), columns = Model, values = γ1 
gamma1_matrix = (
    tidy.pivot_table(
        index=["Ticker","Period",  "Lags"],
        columns="Model",
        values="γ1",
        aggfunc="first"
    )
    .sort_index()
    .sort_index(axis=1)
)

print(gamma1_matrix)

In [ ]:
# jacquard between models

# prepare df for manipulation
sig = VIX_aini_to_ret_sig.copy()

# Create a unique identifier for comparison
key_cols = ["AINI_variant", "Ticker", "Period"]
sig["combo"] = sig[key_cols].astype(str).agg("_".join, axis=1)

# 4) Build a set of combos per model
model_sets = {m: set(g["combo"].unique()) for m, g in sig.groupby("Model")}

# In case there are models with zero significant rows, include them with empty sets:
all_models = sorted(df["Model"].astype(str).unique())
for m in all_models:
    model_sets.setdefault(m, set())

# 5) Construct the n×n Jaccard matrix with diagonal = 1.0
mat = pd.DataFrame(index=all_models, columns=all_models, dtype=float)

for i in all_models:
    Si = model_sets[i]
    for j in all_models:
        if i == j:
            mat.loc[i, j] = 1.0
        else:
            Sj = model_sets[j]
            union = len(Si | Sj)
            inter = len(Si & Sj)
            mat.loc[i, j] = (inter / union) if union > 0 else float("nan")

# Optional: round for readability
jacard_matrix = mat.round(3)

print(jacard_matrix)

In [ ]:
# Drop all-NaN columns
VIX_aini_to_ret_sig = VIX_aini_to_ret_sig.dropna(axis=1, how="all")

# Collapse duplicate "γ1" columns into one Series
g1_block = VIX_aini_to_ret_sig.loc[:, VIX_aini_to_ret_sig.columns == "γ1"]  # may be multiple cols
γ1_series = (
    g1_block.bfill(axis=1).iloc[:, 0]    # first non-null across dup γ1 cols
    .pipe(pd.to_numeric, errors="coerce") # ensure numeric
)

# Add clean γ1, compute abs, sort, slice
VIX_aini_to_ret_sig_sorted = (
    VIX_aini_to_ret_sig
    .assign(γ1_clean=γ1_series)
    .assign(abs_γ1=lambda d: d["γ1_clean"].abs())
    .sort_values("abs_γ1", ascending=False)
)

VIX_aini_to_ret_sig_sort_cut = VIX_aini_to_ret_sig_sorted.iloc[:20].drop(columns=["abs_γ1"])

# Export LaTeX
export_regression_table(
    df=VIX_aini_to_ret_sig_sort_cut,
    title="Granger-Causality, jointly significant results (AINI $\\to$ Returns, controlled for VIX).Top 20 Sorted by $\gamma_1$. \\textit{Source:} Own.",
    output_filename="gc_vix_aini_to_ret_sort_Gamma_cut",
    output_format="tex",
    latex_env="tabular",
    include_caption_label=True,
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",
)



In [ ]:
# investigate sign fllips
def gamma_sign_switch_summary(df: pd.DataFrame) -> dict:
    g1 = pd.to_numeric(df['γ1'], errors='coerce')
    g2 = pd.to_numeric(df['γ2'], errors='coerce')
    g3 = pd.to_numeric(df['γ3'], errors='coerce')

    total_non_nan_12 = df[['γ1', 'γ2']].dropna().shape[0]
    m12 = g1.notna() & g2.notna() & (g1 != 0) & (g2 != 0)
    compared12 = int(m12.sum())
    switches12 = int((g1[m12] * g2[m12] < 0).sum())
    switch_share12 = switches12 / compared12 if compared12 else np.nan
    compared_share12 = compared12 / total_non_nan_12 if total_non_nan_12 else np.nan

    total_non_nan_23 = df[['γ2', 'γ3']].dropna().shape[0]
    m23 = g2.notna() & g3.notna() & (g2 != 0) & (g3 != 0)
    compared23 = int(m23.sum())
    switches23 = int((g2[m23] * g3[m23] < 0).sum())
    switch_share23 = switches23 / compared23 if compared23 else np.nan
    compared_share23 = compared23 / total_non_nan_23 if total_non_nan_23 else np.nan

    return {
        'γ1→γ2_switches': switches12,
        'γ1→γ2_compared': compared12,
        'γ1→γ2_total_non_nan': total_non_nan_12,
        'γ1→γ2_switch_share': switch_share12,
        'γ1→γ2_compared_share': compared_share12,
        'γ2→γ3_switches': switches23,
        'γ2→γ3_compared': compared23,
        'γ2→γ3_total_non_nan': total_non_nan_23,
        'γ2→γ3_switch_share': switch_share23,
        'γ2→γ3_compared_share': compared_share23,
    }

# Example:
result = gamma_sign_switch_summary(VIX_aini_to_ret_sig_sorted)
print(result)
print(result)


In [ ]:
VIX_aini_to_ret_sig_r2 = VIX_aini_to_ret_sig_sorted.copy()

mask = VIX_aini_to_ret_sig_r2['BH empirical p'] < VIX_aini_to_ret_sig_r2['BH analytical p']
df_higher_empirical = VIX_aini_to_ret_sig_r2[mask]

print(f"Cases where BH empirical p < BH analytical p: {mask.sum()}")
df_higher_empirical


In [ ]:
# sort by R^2
VIX_aini_to_ret_sig_r2 = (
    VIX_aini_to_ret_sig
    .assign(γ1_clean=γ1_series)
    .sort_values("adj_r2_u", ascending=False)
)

VIX_aini_to_ret_sig_r2[["Model","AINI_variant","Ticker","Period","adj_r2_u"]]

In [ ]:
# Export LaTeX
export_regression_table(
    df=VIX_aini_to_ret_sig_sorted,
    title="Granger-Causality, jointly significant results (AINI $\\to$ Returns, controlled for VIX).Sorted by absolute magnitude of $\gamma_1$. Source: Own.",
    output_filename="gc_vix_aini_to_ret_sort_Gamma",
    output_format="tex",
    latex_env="tabular",
    include_caption_label=True,
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",
)

In [ ]:
#vix_ret_to_aini_sub
vix_ret_to_aini_sub = vix_ret_to_aini_sub.sort_values("adj_r2_u",ascending=False)
vix_ret_to_aini_sub_cut = vix_ret_to_aini_sub.iloc[0:20]

In [ ]:
# investigate sign flips for beta
def beta_sign_switch_summary(df: pd.DataFrame) -> dict:
    b1 = pd.to_numeric(df['β1'], errors='coerce')
    b2 = pd.to_numeric(df['β2'], errors='coerce')
    b3 = pd.to_numeric(df['β3'], errors='coerce')

    total_non_nan_12 = df[['β1', 'β2']].dropna().shape[0]
    m12 = b1.notna() & b2.notna() & (b1 != 0) & (b2 != 0)
    compared12 = int(m12.sum())
    switches12 = int((b1[m12] * b2[m12] < 0).sum())
    switch_share12 = switches12 / compared12 if compared12 else np.nan
    compared_share12 = compared12 / total_non_nan_12 if total_non_nan_12 else np.nan

    total_non_nan_23 = df[['β2', 'β3']].dropna().shape[0]
    m23 = b2.notna() & b3.notna() & (b2 != 0) & (b3 != 0)
    compared23 = int(m23.sum())
    switches23 = int((b2[m23] * b3[m23] < 0).sum())
    switch_share23 = switches23 / compared23 if compared23 else np.nan
    compared_share23 = compared23 / total_non_nan_23 if total_non_nan_23 else np.nan

    return {
        'β1→β2_switches': switches12,
        'β1→β2_compared': compared12,
        'β1→β2_total_non_nan': total_non_nan_12,
        'β1→β2_switch_share': switch_share12,
        'β1→β2_compared_share': compared_share12,
        'β2→β3_switches': switches23,
        'β2→β3_compared': compared23,
        'β2→β3_total_non_nan': total_non_nan_23,
        'β2→β3_switch_share': switch_share23,
        'β2→β3_compared_share': compared_share23,
    }

# Example:
result_beta = beta_sign_switch_summary(vix_ret_to_aini_sub)
print(result_beta)


In [ ]:
# investigate by r^2
vix_ret_to_aini_sub[["Model","AINI_variant","Ticker","Period","adj_r2_u"]].sort_values("adj_r2_u",ascending=False)

In [ ]:
# Export LaTeX reverse direction
export_regression_table(
    df=vix_ret_to_aini_sub,
    title="Granger-Causality, jointly significant results (Returns $\\to$ AINI, controlled for VIX).Sorted by absolute magnitude of $\beta_1$. Source: Own.",
    output_filename="gc_vix_return_to_aini",
    output_format="tex",
    latex_env="tabular",
    include_caption_label=True,
    reverse=True,
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",
)


In [ ]:
# Export LaTeX reverse direction short
export_regression_table(
    df=vix_ret_to_aini_sub_cut,
    title="Granger-Causality, jointly significant results (Returns $\\to$ AINI, controlled for VIX).Sorted by absolute magnitude of $\beta_1$. Source: Own.",
    output_filename="gc_vix_return_to_aini_cut",
    output_format="tex",
    latex_env="tabular",
    include_caption_label=True,
    reverse=True,
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",
)

In [ ]:
# Export LaTeX
export_regression_table(
    df=VIX_aini_to_ret_sig_sorted,
    title="Granger-Causality, jointly significant results (AINI $\\to$ Returns, controlled for VIX).Sorted by absolute magnitude of $\gamma_1$. Source: Own.",
    output_filename="gc_vix_aini_to_ret_sort_Gamma",
    output_format="tex",
    latex_env="tabular",
    include_caption_label=False,
    coef_digits=3,
    p_digits=3,
    tabcolsep_pt=2.0,
    font_size_cmd="scriptsize",
)

Investigate assets

In [ ]:
# beautify
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].replace({"2023_24": "2023-2024"})
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].replace({"2024_25": "2024-2025"})
vix_aini_to_ret_sub["Period"] = vix_aini_to_ret_sub["Period"].replace({"2023_24_25": "2023-2025"})


In [ ]:
# save merged results
gc_all_results_VIX.to_csv(var_path / "granger_causality_VIX.csv", index=False)

# define table path
table_path = root_dir / "reports" / "tables"

# Export as HTML for online appendix
gc_all_results.to_html(table_path / "granger_causality_VIX.html", index=False)

Controlled for VIX as collider

Investigate results

In [ ]:
variants = ["w0", "w1", "w2", "binary"]
dfs = {}

for v in variants:
    dfs[v] = pd.read_csv(var_path / f"vix_causality_{v}_log_growth_closed.csv")
    dfs[v]["variant"] = v

dfs["w0"]["BH_corr_F_pval_HC3"]

In [ ]:
# load
variants = ["w0", "w1", "w2", "binary"]
dfs = {}

for v in variants:
    dfs[v] = pd.read_csv(var_path / f"vix_causality_{v}_log_growth_closed.csv")
    dfs[v]["Model"] = v if v != "binary" else "c"
    dfs[v]["joint rej. (α=0.1)"] = (dfs[v]["BH_corr_F_pval"] < 0.1) & (dfs[v]["BH_corr_F_pval_HC3"] < 0.1)

# find rejections
rejects = {}
for v in variants:
    df = dfs[v]
    rej = df[(df["BH_corr_F_pval"] < 0.1) & (df["BH_corr_F_pval_HC3"] < 0.1)]          
    rejects[v] = rej
    print(rej.head())

# unpack
all_w0 , all_w1, all_w2, all_custom = dfs.values()
rej_w0 , rej_w1, rej_w2, rej_custom = rejects.values()

all_w0

In [ ]:
# find AINI -> VIX
aini_rejects = dict()

for v in variants:
    df = dfs[v]
    rej_aini = df[(df["BH_corr_F_pval"] < 0.1) & (df["BH_corr_F_pval_HC3"] < 0.1) & (df["Direction"] == "AINI_to_VIX")]          
    aini_rejects[v] = rej_aini
    print(rej_aini.head())

all_w0.columns

In [ ]:
# beautify to report
rename_map = {
    "p_x": "Lags",
    "BH_corr_F_pval": "BH empirical p",
    "BH_corr_F_pval_HC3": "BH analytical p",
    "Year": "Period",
    "RET_1" : "β1", 
    "RET_2" : "β2",
    "RET_3" : "β3",
    "AINI_1" : "γ1",
    "AINI_2" : "γ2",
    "AINI_3" : "γ3",
    "VIX_ar_1" : "ζ1",
    "VIX_ar_2" : "ζ2",
    "VIX_ar_3" : "ζ3"
}

aini_map = {
    "EMA_02" : "EMA^{0.2}",
    "EMA_08" : "EMA^{0.8}",
    "normalized_AINI" : "AINI^{norm}"
}


# rename for reporting
for name, df in dfs.items():
    # rename columns
    df = df.rename(columns=rename_map)

    # ensure Period is string for robust comparisons
    df["Period"] = df["Period"].astype(str)

    # pretty-print values for the AINI variant (this changes the actual values)
    df["AINI_variant"] = df["AINI_variant"].replace(aini_map)

    # build LaTeX-ready index label
    df["Measure"] = "$" + df["AINI_variant"] + "_{" + df["Model"] + "}$"

    dfs[name] = df


# filter non-stationarity measures, i.e. windows in 2025; EMA_{0.2} in 2025 for costum
for name, df in dfs.items():

    # keep only 'custom' for 2025, others exclude 2025
    df = df[(df["Model"] == "custom") | (df["Period"] != "2025")]
    
    # exclude EMA_02 in 2025
    df = df[~((df["AINI_variant"] == "EMA_02") & (df["Period"] == "2025"))]
    
    # reassign back
    dfs[name] = df  

# group by direction; exrtact dfs with same direction
direction_dfs = {}

for name, df in dfs.items():
    for direction, subdf in df.groupby("Direction"):
        direction_dfs[f"{name}_{direction}"] = subdf

# concat with same direction
direction_groups = {}

for key, df in direction_dfs.items():
    direction = df["Direction"].iloc[0]  # same for all rows in that df

    if direction not in direction_groups:
        direction_groups[direction] = []
    direction_groups[direction].append(df.assign(Model=key.split("_")[0]))

# Concatenate all per direction
combined_by_direction = {
    direction: pd.concat(dfs, ignore_index=True)
    for direction, dfs in direction_groups.items()
}

AINI_to_VIX, Return_to_VIX = combined_by_direction.values()
AINI_to_VIX

In [ ]:
# load data to test for covariance
log_vix = pd.read_csv(var_path / "log_growth_VIX.csv")
aini_custom = pd.read_csv(var_path / "binary_AINI_variables.csv")
aini_w0 = pd.read_csv(var_path / "w0_AINI_variables.csv")
aini_w1 = pd.read_csv(var_path / "w1_AINI_variables.csv")
aini_w2 =  pd.read_csv(var_path / "w2_AINI_variables.csv")

merged, tidy, pivot, extrema = compute_aini_extrema(aini_w0,aini_w1,aini_w2,aini_custom,
                                                    )
merged.columns

In [ ]:
#  Align and merge data 
aini_measures = [
    'EMA_08_w0', 'normalized_AINI_w1', 'EMA_02_w1',
    'EMA_08_w1', 'normalized_AINI_w2', 'EMA_02_w2',
    'EMA_08_w2', 'normalized_AINI_custom', 
    'EMA_02_custom', 'EMA_08_custom'
]


# Ensure datetime
log_vix['date'] = pd.to_datetime(log_vix['date'])
merged['date'] = pd.to_datetime(merged['date'])

# Merge both dataframes by date
df = pd.merge(
    log_vix[['date', 'log_growth_closed']],
    merged[['date'] + aini_measures],
    on='date',
    how='inner'
)

# Add year column
df['year'] = df['date'].dt.year

periods = {
    '2023': (2023, 2023),
    '2024': (2024, 2024),
    '2025': (2025, 2025),
    '2023–2024': (2023, 2024),
    '2024–2025': (2024, 2025),
    '2023-2025': (2023, 2025)
}

# Compute correlations
results = []
for label, (start, end) in periods.items():
    subset = df[(df['year'] >= start) & (df['year'] <= end)]
    for var in aini_measures:
        valid = subset[['log_growth_closed', var]].dropna()
        corr = pearsonr(valid['log_growth_closed'], valid[var])[0] if len(valid) > 1 else None
        results.append({'period': label, 'measure': var, 'corr': corr})

corr_df = pd.DataFrame(results).pivot(index='measure', columns='period', values='corr')
corr_df = corr_df.round(3).sort_index()

#  Display 
print(corr_df)

In [ ]:
# Variables
aini_measures = [
    'EMA_08_w0', 'normalized_AINI_w1', 'EMA_02_w1',
    'EMA_08_w1', 'normalized_AINI_w2', 'EMA_02_w2',
    'EMA_08_w2', 'normalized_AINI_custom', 
    'EMA_02_custom', 'EMA_08_custom'
]

# Ensure datetime
log_vix['date'] = pd.to_datetime(log_vix['date'])
merged['date'] = pd.to_datetime(merged['date'])

# Merge on date
df = pd.merge(
    log_vix[['date', 'log_growth_closed']],
    merged[['date'] + aini_measures],
    on='date',
    how='inner'
)

# Add year column
df['year'] = df['date'].dt.year

# Define periods
periods = {
    '2023': (2023, 2023),
    '2024': (2024, 2024),
    '2025': (2025, 2025),
    '2023–2024': (2023, 2024),
    '2024–2025': (2024, 2025),
    '2023–2025': (2023, 2025)
}

# --- Compute covariances ---
results = []
for label, (start, end) in periods.items():
    subset = df[(df['year'] >= start) & (df['year'] <= end)]
    for var in aini_measures:
        valid = subset[['log_growth_closed', var]].dropna()
        cov = valid['log_growth_closed'].cov(valid[var]) if len(valid) > 1 else None
        results.append({'period': label, 'measure': var, 'cov': cov})

# --- Build covariance table ---
cov_df = pd.DataFrame(results).pivot(index='measure', columns='period', values='cov')
cov_df = cov_df.round(6).sort_index()

# --- Display ---
print(cov_df)


In [ ]:
def diff_stats_by_period(df: pd.DataFrame) -> pd.DataFrame:

    #  Ensure proper dtypes & ordering
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'], utc=False)
    df = df.sort_values(['Ticker', 'Date'])
    df['Year'] = df['Date'].dt.year

    # Define periods
    periods = {
        '2023': [2023],
        '2024': [2024],
        '2025': [2025],
        '2023–2024': [2023, 2024],
        '2023–2025': [2023, 2024, 2025],
        '2024–2025': [2024, 2025],
    }

    out = []
    for label, years in periods.items():
        sub = df[df['Year'].isin(years)].copy()

        # Compute log differences per Ticker
        sub['log_diff'] = sub.groupby('Ticker')['Adj Close'].transform(lambda x: np.log(x).diff())

        # --- Aggregate stats 
        agg = (
            sub.dropna(subset=['log_diff'])
               .groupby('Ticker', as_index=False)['log_diff']
               .agg(mean='mean', std='std', n='size')
        )
        agg.insert(1, 'Period', label)
        out.append(agg)

    # Combine results
    result = pd.concat(out, ignore_index=True)

    # Sort output
    period_order = list(periods.keys())
    result['Period'] = pd.Categorical(result['Period'], categories=period_order, ordered=True)
    result = result.sort_values(['Ticker', 'Period']).reset_index(drop=True)
    return result


#  usage
df = pd.read_csv(root_dir / "data" / "raw" / "financial" / "full_daily_2023_2025.csv")

stats = diff_stats_by_period(df)

for row in range(len(stats)):
    print(stats.iloc[row].to_dict())
